<a href="https://colab.research.google.com/github/ashwinperti/movie_review1/blob/main/Sentiment_Analysis_with_Sklearn.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
import numpy as np
!git clone https://github.com/laxmimerit/IMDB-Movie-Reviews-Large-Dataset-50k

Cloning into 'IMDB-Movie-Reviews-Large-Dataset-50k'...
remote: Enumerating objects: 10, done.
remote: Counting objects: 100% (10/10), done.
remote: Compressing objects: 100% (8/8), done.
remote: Total 10 (delta 1), reused 0 (delta 0), pack-reused 0
Unpacking objects: 100% (10/10), done.


In [2]:
df = pd.read_excel("/content/IMDB-Movie-Reviews-Large-Dataset-50k/train.xlsx") 

df.shape ## (25000, 2)
df.head() 

,Reviews,Sentiment
0,"When I first tuned in on this morning news, I ...",neg
1,"Mere thoughts of ""Going Overboard"" (aka ""Babes...",neg
2,Why does this movie fall WELL below standards?...,neg
3,Wow and I thought that any Steven Segal movie ...,neg
4,"The story is seen before, but that does'n matt...",neg


In [3]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.svm import LinearSVC
from sklearn.metrics import classification_report

!pip install git+https://github.com/laxmimerit/preprocess_kgptalkie.git --upgrade --force-reinstall

  Cloning https://github.com/laxmimerit/preprocess_kgptalkie.git to /tmp/pip-req-build-ighyrd22
  Running command git clone -q https://github.com/laxmimerit/preprocess_kgptalkie.git /tmp/pip-req-build-ighyrd22
  Created wheel for preprocess-kgptalkie: filename=preprocess_kgptalkie-0.1.3-py3-none-any.whl size=11756 sha256=1a24aced5f738c09dead6690b9da6a7700f1cdf6d49c642b6877f6e092b83302
  Stored in directory: /tmp/pip-ephem-wheel-cache-v9gt09pe/wheels/0d/b3/29/bfe3deffda68980088d17b81331be6667e837ffb4a071bae82
Successfully built preprocess-kgptalkie


In [4]:
import preprocess_kgptalkie as ps
import re
def get_clean(x):
    x = str(x).lower().replace('\\', '').replace('_', ' ')
    x = ps.cont_exp(x)
    x = ps.remove_emails(x)
    x = ps.remove_urls(x)
    x = ps.remove_html_tags(x)
    x = ps.remove_rt(x)
    x = ps.remove_accented_chars(x)
    x = ps.remove_special_chars(x)
    x = re.sub("(.)\\1{2,}", "\\1", x)
    return x



In [5]:
df['Reviews'] = df['Reviews'].apply(lambda x: get_clean(x))
df.head()

,Reviews,Sentiment
0,when i first tuned in on this morning news i t...,neg
1,mere thoughts of going overboard aka babes aho...,neg
2,why does this movie fall well below standards ...,neg
3,wow and i thought that any steven segal movie ...,neg
4,the story is seen before but that doesand matt...,neg


In [17]:
tfid = TfidfVectorizer(max_features=5000)
X = df['Reviews']
y = df['Sentiment']

X = tfid.fit_transform(X)
X

<25000x5000 sparse matrix of type '<class 'numpy.float64'>'
	with 2845641 stored elements in Compressed Sparse Row format>

In [11]:
clf = LinearSVC()
X_train, X_test, y_train,y_test = train_test_split(X,y, test_size=0.2, random_state=0) #to reevaluate the same with these settings
clf.fit(X_train, y_train)


LinearSVC()

In [12]:
y_pred = clf.predict(X_test)

In [13]:
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

         neg       0.87      0.87      0.87      2480
         pos       0.87      0.88      0.88      2520

    accuracy                           0.87      5000
   macro avg       0.87      0.87      0.87      5000
weighted avg       0.87      0.87      0.87      5000



In [18]:
x = "this movie is very good. thanks a lot for making isnt"
x = get_clean(x)
vec_x = tfid.transform([x])
print(vec_x.shape)
clf.predict(vec_x)

(1, 5000)


array(['pos'], dtype=object)

In [21]:
import pickle
pickle.dump(clf, open('model', 'wb'))